<a href="https://colab.research.google.com/github/Carlo-Cascini/Carlo-Cascini/blob/main/Copia_di_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import csv

# Funzione per ottenere i simboli delle azioni dal sito web Wikipedia dell'indice S&P 500
def get_sp500_symbols():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable sortable'})
    symbols = []
    for row in table.findAll('tr')[1:]:
        symbol = row.findAll('td')[0].text.strip()
        symbols.append(symbol)
    return symbols

# Ottieni i simboli delle azioni del S&P 500
sp500_symbols = get_sp500_symbols()

# Definiamo una funzione per l'encoding dei dati di ogni azione
def encode_stock_data(stock_symbol):
    # Scarica i dati storici delle azioni
    data = yf.download(stock_symbol, start="2020-01-01", end="2022-01-01", progress=False)
    prices = data["Close"].values.astype('float32')

    # Normalizzazione dei dati
    mean_price = np.mean(prices)
    std_price = np.std(prices)
    prices = (prices - mean_price) / std_price

    # Converti i dati in un tensore PyTorch
    prices_tensor = torch.tensor(prices).unsqueeze(1)  # Aggiunge una dimensione per fare il batch

    # Definisci il modello di encoder
    input_size = 1  # Un solo feature (prezzo di chiusura)
    hidden_size = 64  # Dimensione dello stato nascosto LSTM
    latent_size = 32  # Dimensione dello spazio latente
    num_layers = 2   # Numero di strati LSTM
    model = Encoder(input_size, hidden_size, latent_size, num_layers).to(device)

    # Eseguiamo l'encoding dei dati
    with torch.no_grad():  # Disabilitiamo il calcolo del gradiente per l'encoding
        prices_tensor = prices_tensor.to(device)
        encoded_data = model(prices_tensor)

    return encoded_data

# Definiamo un modello complesso per l'encoder
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size, num_layers):
        super(Encoder, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, latent_size)

    def forward(self, x):
        _, (hidden, _) = self.rnn(x)
        return self.fc(hidden[-1])

# Verifica disponibilità GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo disponibile:", device)

# Creiamo un dizionario per memorizzare i dati encodati per ciascuna azione
encoded_data_dict = {}

# Iteriamo su ogni azione nel S&P 500
for symbol in sp500_symbols:
    print("Encoding per", symbol)
    try:
        encoded_data = encode_stock_data(symbol)
        encoded_data_dict[symbol] = encoded_data.cpu().numpy()
    except Exception as e:
        print("Errore durante l'encoding di", symbol, ":", e)

# Salva i dati encodati in un file CSV
csv_file = "encoded_stock_data.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Symbol", "Encoded Data"])
    for symbol, data in encoded_data_dict.items():
        writer.writerow([symbol, data])

print("Dati encodati salvati in", csv_file)


Dispositivo disponibile: cpu
Encoding per MMM
Encoding per AOS
Encoding per ABT
Encoding per ABBV
Encoding per ACN
Encoding per ADBE
Encoding per AMD
Encoding per AES
Encoding per AFL
Encoding per A
Encoding per APD
Encoding per ABNB
Encoding per AKAM
Encoding per ALB
Encoding per ARE
Encoding per ALGN
Encoding per ALLE
Encoding per LNT
Encoding per ALL
Encoding per GOOGL
Encoding per GOOG
Encoding per MO
Encoding per AMZN
Encoding per AMCR
Encoding per AEE
Encoding per AAL
Encoding per AEP
Encoding per AXP
Encoding per AIG
Encoding per AMT
Encoding per AWK
Encoding per AMP
Encoding per AME
Encoding per AMGN
Encoding per APH
Encoding per ADI
Encoding per ANSS
Encoding per AON
Encoding per APA
Encoding per AAPL
Encoding per AMAT
Encoding per APTV
Encoding per ACGL
Encoding per ADM
Encoding per ANET
Encoding per AJG
Encoding per AIZ
Encoding per T
Encoding per ATO
Encoding per ADSK
Encoding per ADP
Encoding per AZO
Encoding per AVB
Encoding per AVY
Encoding per AXON
Encoding per BKR
Enco

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Errore durante l'encoding di BRK.B : Expected sequence length to be larger than 0 in RNN
Encoding per BBY
Encoding per BIO
Encoding per TECH
Encoding per BIIB
Encoding per BLK
Encoding per BX
Encoding per BA
Encoding per BKNG
Encoding per BWA
Encoding per BXP
Encoding per BSX
Encoding per BMY
Encoding per AVGO
Encoding per BR


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2020-01-01 -> 2022-01-01)')


Encoding per BRO
Encoding per BF.B
Errore durante l'encoding di BF.B : Expected sequence length to be larger than 0 in RNN


...

Errore durante l'encoding di GEHC : Expected sequence length to be larger than 0 in RNN
Encoding per GEN
Encoding per GNRC
Encoding per GD
Encoding per GE
Encoding per GIS
Encoding per GM
Encoding per GPC
Encoding per GILD
Encoding per GPN
Encoding per GL
Encoding per GS
Encoding per HAL
Encoding per HIG
Encoding per HAS
Encoding per HCA
Encoding per DOC
Encoding per HSIC
Encoding per HSY
Encoding per HES
Encoding per HPE
Encoding per HLT
Encoding per HOLX
Encoding per HD
Encoding per HON
Encoding per HRL
Encoding per HST
Encoding per HWM
Encoding per HPQ
Encoding per HUBB
Encoding per HUM
Encoding per HBAN
Encoding per HII
Encoding per IBM
Encoding per IEX
Encoding per IDXX
Encoding per ITW
Encoding per ILMN
Encoding per INCY
Encoding per IR
Encoding per PODD
Encoding per INTC
Encoding per ICE
Encoding per IFF
Encoding per IP
Encoding per IPG
Encoding per INTU
Encoding per ISRG
Encoding per IVZ
Encoding per INVH
Encoding per IQV
Encoding per IRM
Encoding per JBHT
Encoding per JBL
Enco

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 1577854800, endDate = 1641013200")
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Errore durante l'encoding di KVUE : Expected sequence length to be larger than 0 in RNN
Encoding per KDP
Encoding per KEY
Encoding per KEYS
Encoding per KMB
Encoding per KIM
Encoding per KMI
Encoding per KLAC
Encoding per KHC
Encoding per KR
Encoding per LHX
Encoding per LH
Encoding per LRCX
Encoding per LW
Encoding per LVS
Encoding per LDOS
Encoding per LEN
Encoding per LIN
Encoding per LYV
Encoding per LKQ
Encoding per LMT
Encoding per L
Encoding per LOW
Encoding per LULU
Encoding per LYB
Encoding per MTB
Encoding per MRO
Encoding per MPC
Encoding per MKTX
Encoding per MAR
Encoding per MMC
Encoding per MLM
Encoding per MAS
Encoding per MA
Encoding per MTCH
Encoding per MKC
Encoding per MCD
Encoding per MCK
Encoding per MDT
Encoding per MRK
Encoding per META
Encoding per MET
Encoding per MTD
Encoding per MGM
Encoding per MCHP
Encoding per MU
Encoding per MSFT
Encoding per MAA
Encoding per MRNA
Encoding per MHK
Encoding per MOH
Encoding per TAP
Encoding per MDLZ
Encoding per MPWR
Encod

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1577854800, endDate = 1641013200")
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Errore durante l'encoding di VLTO : Expected sequence length to be larger than 0 in RNN
Encoding per VRSN
Encoding per VRSK
Encoding per VZ
Encoding per VRTX
Encoding per VFC
Encoding per VTRS
Encoding per VICI
Encoding per V
Encoding per VMC
Encoding per WRB
Encoding per WAB
Encoding per WBA
Encoding per WMT
Encoding per DIS
Encoding per WBD
Encoding per WM
Encoding per WAT
Encoding per WEC
Encoding per WFC
Encoding per WELL
Encoding per WST
Encoding per WDC
Encoding per WRK
Encoding per WY
Encoding per WMB
Encoding per WTW
Encoding per GWW
Encoding per WYNN
Encoding per XEL
Encoding per XYL
Encoding per YUM
Encoding per ZBRA
Encoding per ZBH
Encoding per ZTS
Dati encodati salvati in encoded_stock_data.csv
